In [1]:
!pip install scikit-learn

In [2]:
import pandas as pd

from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction.text import CountVectorizer

import spacy


In [38]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
train = train.dropna()

In [39]:
print(train.shape)
train.head()

(2476, 7)


,id,author,description,price,ratingValue,pert_alcohol,category
0,1,John Hansell,A marriage of 13 and 18 year old bourbons. A m...,85.0,97,51.50,2.0
1,2,Dave Broom,There have been some legendary Bowmores from t...,13500.0,97,42.90,1.0
2,3,John Hansell,This bottling celebrates master distiller Park...,150.0,97,50.00,2.0
3,4,John Hansell,What impresses me most is how this whisky evol...,4500.0,97,40.50,1.0
5,9,Fred Minnick,"A caramel-laden fruit bouquet, followed by une...",150.0,96,54.49,2.0


In [40]:
print(test.shape)
test.head()

(288, 6)


,id,author,description,price,ratingValue,pert_alcohol
0,955,Fred Minnick,"Think carnival aromas—the good ones, anyway—me...",36.0,90,50.0
1,3532,Lew Bryson,"A blend of three bourbons, between 6 and 12 ye...",90.0,82,49.3
2,1390,Davin de Kergommeaux,"The nose is focused on cereal, hints of fresh ...",48.0,89,45.0
3,1024,Gavin Smith,Swiss-based Chapter 7 released this 19 year ol...,180.0,90,55.8
4,1902,Gavin Smith,Valkyrie replaces the current Dark Origins exp...,71.0,87,45.9


In [41]:
# pick features
features = train.columns.drop(['id','category'])
target = 'category'

# train and test
X_train = train[features]
X_test = test[features]

y_train = train[target]

In [42]:
encoder = OrdinalEncoder()

X_train['author'] = encoder.fit_transform(train['author'].values.reshape(-1,1))
X_test['author'] = encoder.transform(test['author'].values.reshape(-1,1))
# encoder.transform(train['author'].values.reshape(-1,1))

/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [43]:
X_train.head()

,author,description,price,ratingValue,pert_alcohol
0,8.0,A marriage of 13 and 18 year old bourbons. A m...,85.0,97,51.50
1,1.0,There have been some legendary Bowmores from t...,13500.0,97,42.90
2,8.0,This bottling celebrates master distiller Park...,150.0,97,50.00
3,8.0,What impresses me most is how this whisky evol...,4500.0,97,40.50
5,4.0,"A caramel-laden fruit bouquet, followed by une...",150.0,96,54.49


In [44]:
# description
nlp = spacy.load('en_core_web_lg')

In [45]:
def tokenize(doc):
    d = nlp(doc)
    tokens = []
    
    # You can extract anything you want from Spacy to use as a feature
    # Noun Chunks aren't the end all be all.
    # Try Extracting just tokens, or just adjectives, or just pronouns
    for chunk in d.noun_chunks:
        tokens.append(chunk.lemma_)
        
    return tokens

vect = CountVectorizer(analyzer=tokenize, max_df=.95, min_df=.05)

In [47]:
vect.fit(X_train['description'])

CountVectorizer(analyzer=<function tokenize at 0x1a275b1ae8>, binary=False,
        decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=0.95,
        max_features=None, min_df=0.05, ngram_range=(1, 1),
        preprocessor=None, stop_words=None, strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, vocabulary=None)

In [48]:
vect.get_feature_names()

['-PRON-',
 'a hint',
 'caramel',
 'cinnamon',
 'ginger',
 'hint',
 'honey',
 'malt',
 'note',
 'nutmeg',
 'oak',
 'smoke',
 'spice',
 'the finish',
 'the mouth',
 'the nose',
 'the palate',
 'this whisky',
 'toffee',
 'vanilla',
 'water']

In [49]:
X = vect.transform(X_train['description'])
X = X.todense()
X

matrix([[0, 0, 0, ..., 0, 0, 1],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [2, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0]])

In [50]:
X = pd.DataFrame(X, columns = vect.get_feature_names())

In [51]:
X_train.shape, X.shape

((2476, 5), (2476, 21))

In [71]:
X_train.head()

,author,description,price,ratingValue,pert_alcohol,id
0,8.0,A marriage of 13 and 18 year old bourbons. A m...,85.0,97,51.50,0
1,1.0,There have been some legendary Bowmores from t...,13500.0,97,42.90,1
2,8.0,This bottling celebrates master distiller Park...,150.0,97,50.00,2
3,8.0,What impresses me most is how this whisky evol...,4500.0,97,40.50,3
5,4.0,"A caramel-laden fruit bouquet, followed by une...",150.0,96,54.49,4


In [69]:
X_train['id'] = list(range(2476))
X['id'] = list(range(2476))

/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [70]:
X.head()

,-PRON-,a hint,caramel,cinnamon,ginger,hint,honey,malt,note,nutmeg,...,spice,the finish,the mouth,the nose,the palate,this whisky,toffee,vanilla,water,id
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
3,3,1,0,0,0,0,0,0,1,0,...,0,0,0,1,1,1,0,0,0,3
4,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [72]:
X.isna().sum()

-PRON-         0
a hint         0
caramel        0
cinnamon       0
ginger         0
hint           0
honey          0
malt           0
note           0
nutmeg         0
oak            0
smoke          0
spice          0
the finish     0
the mouth      0
the nose       0
the palate     0
this whisky    0
toffee         0
vanilla        0
water          0
id             0
dtype: int64

In [36]:
rfc = RandomForestClassifier()

In [37]:
rfc.fit(X_train, y_train)

/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ValueError: Number of labels=2476 does not match number of samples=2113

In [76]:
matrix_df = pd.DataFrame(X)
matrix_df.head(50)

,0,1,2,3,4,5,6,7,8,9,...,720,721,722,723,724,725,726,727,728,729
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
matrix_df = pd.DataFrame(X)

columns = [0,1,2,3,4,5,6,7,8,9]
matrix_df = matrix_df[columns]

matrix_df['id'] = train.id
X_train['id'] = train.id


In [82]:
X_train = X_train.merge(matrix_df, on='id')
X_train = X_train.drop(columns='id')
X_train.head()

,author,price,ratingValue,pert_alcohol,description,0,1,2,3,4,5,6,7,8,9
0,8.0,85.0,97,51.50,A marriage of 13 and 18 year old bourbons. A m...,0,0,0,0,0,0,0,0,0,0
1,1.0,13500.0,97,42.90,There have been some legendary Bowmores from t...,0,0,0,0,0,0,0,0,0,0
2,8.0,150.0,97,50.00,This bottling celebrates master distiller Park...,0,0,0,0,0,0,0,0,0,0
3,8.0,4500.0,97,40.50,What impresses me most is how this whisky evol...,0,0,0,0,0,0,0,0,0,0
4,4.0,150.0,96,54.49,"A caramel-laden fruit bouquet, followed by une...",0,0,0,0,0,0,0,0,0,0


In [71]:
len(X)

2476

In [70]:
len(X)

2476

In [45]:
# vect.transform(X_train['description'])
# X_train['description'] = vect.transform(X_train['description'])

//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,author,price,ratingValue,pert_alcohol
0,8.0,85.0,97,51.50
1,1.0,13500.0,97,42.90
2,8.0,150.0,97,50.00
3,8.0,4500.0,97,40.50
5,4.0,150.0,96,54.49


In [46]:
# vect.transform(X_test['description'])
# X_test['description'] = vect.transform(X_test['description'])

//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [56]:
rfc = RandomForestClassifier()

rfc.fit(X_train, y_train)

//anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ValueError: setting an array element with a sequence.

In [48]:
X_train

,author,description,price,ratingValue,pert_alcohol
0,8.0,"(1, 0)\t1\n (1, 1)\t1\n (2, 0)\t1\n (2, 1...",85.0,97,51.50
1,1.0,"(1, 0)\t1\n (1, 1)\t1\n (2, 0)\t1\n (2, 1...",13500.0,97,42.90
2,8.0,"(1, 0)\t1\n (1, 1)\t1\n (2, 0)\t1\n (2, 1...",150.0,97,50.00
3,8.0,"(1, 0)\t1\n (1, 1)\t1\n (2, 0)\t1\n (2, 1...",4500.0,97,40.50
5,4.0,"(1, 0)\t1\n (1, 1)\t1\n (2, 0)\t1\n (2, 1...",150.0,96,54.49


In [49]:
X_train.description[0]

<2476x2 sparse matrix of type '<class 'numpy.int64'>'
	with 2271 stored elements in Compressed Sparse Row format>

In [ ]:
# make predictions
y_preds = rfc.predict(X_test)
y_preds = [int(i) for i in y_preds]
assert len(y_preds) == len(test)

# create submission csv

sample_submission = pd.read_csv('DS-Unit-4-Sprint-1-NLP/module3-document-classification/data/sample_submission.csv')
sample_submission['category'] = y_preds
sample_submission.to_csv('team_whiskey_submission.csv', index=False)